In [25]:
import tensorflow as tf

import tensorflow_hub as hub

import tensorflow_text as text

In [2]:
from google.colab import files
import pandas as pd

In [6]:
uploaded = files.upload()

Saving spam.xlsx to spam.xlsx


In [7]:
df = pd.read_excel('spam.xlsx')


In [9]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [11]:
df_spam=df[df['Category']=='spam']

df_spam.shape

(747, 2)

In [12]:
df_ham=df[df['Category']=='ham']
df_ham.shape

(4825, 2)

In [13]:
df_ham_downsampled=df_ham.sample(747)

In [14]:
df_balanced=pd.concat([df_spam,df_ham_downsampled])

In [15]:
df_balanced['Category'].value_counts()

,count
Category,
spam,747
ham,747


In [16]:
df_balanced.head()

,Category,Message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [17]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)

In [18]:
df_balanced.head()

,Category,Message,spam
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",1


In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

In [21]:
X_train.head()

,Message
2453,Okay. I've seen it. So i should pick it on fri...
50,What you thinked about me. First time you saw ...
2765,Open rebtel with firefox. When it loads just p...
5449,"Latest News! Police station toilet stolen, cop..."
3409,Do you want a New Nokia 3510i colour phone Del...


In [26]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [27]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']



In [28]:
get_sentence_embeding([
    "500$ discount. hurry up Amazon ",
    "Susnata  are you goinmg to join a Big 4 ?",
    "whats up bro"]
)   #### this will be a 768 length

<tf.Tensor: shape=(3, 768), dtype=float32, numpy=
array([[-0.75503427, -0.44127685, -0.7630213 , ..., -0.45742294,
        -0.7395691 ,  0.7981382 ],
       [-0.81126106, -0.4325406 , -0.92623365, ..., -0.85562897,
        -0.6285386 ,  0.8572137 ],
       [-0.94123036, -0.41131613, -0.8203607 , ..., -0.6303005 ,
        -0.6938779 ,  0.95259076]], dtype=float32)>

In [29]:
e = get_sentence_embeding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates",
    "deloitte",
    "pwc"
]
)

In [30]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[6]],[e[7]])

array([[0.92095083]], dtype=float32)

In [31]:
####BERT

In [32]:
# Load preprocessing and encoder
bert_preprocess = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
)
bert_encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
)


In [33]:
def get_bert_embeddings(text_list):
    preprocessed = bert_preprocess(text_list)
    return bert_encoder(preprocessed)["pooled_output"].numpy()

In [34]:
X_train_embed = get_bert_embeddings(X_train)

In [35]:
X_test_embed = get_bert_embeddings(X_test)

In [36]:
ann_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(768,)),   # 768 from BERT pooled output
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

ann_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [46]:
# Step 3: Traiing my model
ann_model.fit(X_train_embed, y_train, validation_data=(X_test_embed, y_test), epochs=25, batch_size=32)


Epoch 1/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9718 - loss: 0.0834 - val_accuracy: 0.9439 - val_loss: 0.1705
Epoch 2/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9668 - loss: 0.1063 - val_accuracy: 0.9412 - val_loss: 0.1513
Epoch 3/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9650 - loss: 0.1050 - val_accuracy: 0.9492 - val_loss: 0.1467
Epoch 4/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9729 - loss: 0.0857 - val_accuracy: 0.9278 - val_loss: 0.1758
Epoch 5/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9750 - loss: 0.0937 - val_accuracy: 0.9225 - val_loss: 0.1921
Epoch 6/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9690 - loss: 0.1136 - val_accuracy: 0.9465 - val_loss: 0.1520
Epoch 7/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9675 - loss: 0.1008 - val_accuracy: 0.9519 - val_loss: 0.1547
Epoch 8/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9627 - loss: 0.1466 - val_accuracy: 0.9519 - val_loss:

In [38]:
preds = ann_model.predict(X_test_embed)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [47]:
loss, acc = ann_model.evaluate(X_test_embed, y_test)
print(f"Test Accuracy: {acc:.4f}")
print(f"Test Loss: {loss:.4f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9413 - loss: 0.1572 
Test Accuracy: 0.9519
Test Loss: 0.1534


In [40]:
from sklearn.metrics import classification_report, confusion_matrix

# Predictions as binary labels
y_pred_prob = ann_model.predict(X_test_embed)
y_pred = (y_pred_prob > 0.5).astype(int)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
              precision    recall  f1-score   support

           0       0.94      0.90      0.92       187
           1       0.91      0.94      0.92       187

    accuracy                           0.92       374
   macro avg       0.92      0.92      0.92       374
weighted avg       0.92      0.92      0.92       374

[[169  18]
 [ 11 176]]


In [52]:
new_texts = [
    "Congratulations you have won a $1000 dolar amazon gift card,last chance to grab click this link and add bank details",
    "Are we still meeting tomorrow at 5?",
    "You are gorgeous",
    "you got free deloitte gift card"



]

# Convert to embeddings
new_embeddings = get_bert_embeddings(new_texts)

# Predicting
pred_probs = ann_model.predict(new_embeddings)

for text, prob in zip(new_texts, pred_probs):
    label = "SPAM" if prob > 0.5 else "HAM"
    print(f"{text}\nPrediction: {label} ({prob[0]:.4f})\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
Congratulations you have won a $1000 dolar amazon gift card,last chance to grab click this link and add bank details
Prediction: SPAM (0.9922)

Are we still meeting tomorrow at 5?
Prediction: HAM (0.0003)

You are gorgeous
Prediction: HAM (0.0013)

you got free deloitte gift card
Prediction: SPAM (0.5006)

